In [15]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [16]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [17]:
subject = 'Alagoas - Consumo de Cimento (t)'
split_index = 191 #Referente aos 230 anos de input  
train_split = split_index + 1 -30

In [18]:
data = pd.read_csv('2003_mo_model_input_AL.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop(['NFSP - Porcentagem do PIB (%)'], axis=1)
data['Unnamed: 0'] = data['Unnamed: 0'].str[5:].astype(float)
# pd.set_option('display.max_rows', None)
data

,Unnamed: 0,Alagoas - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Alagoas - value,Alagoas - Desemprego,Alagoas - Produção de Cimento (t),Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado,Alagoas - Consumo de Cimento (t)
0,1.0,0.635160,0.724032,-5331.049150,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,0.294434,8.765721,30.779440,2.590528e+07,1.303625e+06,7.576443,2.359888e+07,30.109
1,2.0,0.635571,0.690297,-5318.079644,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,0.296211,8.760196,30.599132,2.593685e+07,1.305020e+06,7.578654,2.360861e+07,27.684
2,3.0,0.635982,0.669681,-5436.417870,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,0.297521,8.754671,30.545910,2.596843e+07,1.306414e+06,7.580866,2.361833e+07,26.534
3,4.0,0.636393,0.660494,-5707.015274,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,0.298766,8.749145,30.377624,2.600000e+07,1.307809e+06,7.583078,2.362806e+07,25.109
4,5.0,0.636804,0.648337,-5599.317941,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,0.299945,8.743620,30.464900,2.603157e+07,1.309203e+06,7.585289,2.363779e+07,25.171
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.587645,NaN,24.556360,NaN,NaN,NaN,NaN,46.546
236,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.587084,NaN,24.501314,NaN,NaN,NaN,NaN,47.781
237,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.585891,NaN,24.609100,NaN,NaN,NaN,NaN,51.171
238,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.584366,NaN,24.538488,NaN,NaN,NaN,NaN,49.126


In [19]:
input_data = data.iloc[:split_index + 1,0:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Unnamed: 0,Alagoas - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Alagoas - value,Alagoas - Desemprego,Alagoas - Produção de Cimento (t),Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado
0,-1.593255,-2.106195,2.723741,0.364808,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.400856,-0.877266,-0.885888,-1.644358,-1.297696,-1.913990,-1.857572
1,-1.303572,-2.072296,2.350880,0.368956,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.374741,-0.879298,-0.900149,-1.627544,-1.242142,-1.888746,-1.832102
2,-1.013890,-2.038397,2.123016,0.331110,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.355481,-0.881330,-0.904358,-1.610731,-1.186588,-1.863502,-1.806633
3,-0.724207,-2.004498,2.021477,0.244570,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.337169,-0.883362,-0.917668,-1.593917,-1.131034,-1.838257,-1.781163
4,-0.434524,-1.970599,1.887113,0.279013,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.319844,-0.885394,-0.910766,-1.577104,-1.075480,-1.813013,-1.755694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.434524,1.358381,-2.010387,2.953658,-1.281958,0.589021,1.365911,0.389193,-1.749976,0.990315,1.043483,-1.575976,1.179374,-1.254203,1.153065,1.054229
188,0.724207,1.351367,-1.870713,3.351323,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.000562,1.037328,-1.527002,1.172432,-1.239868,1.147901,1.048854
189,1.013890,1.344353,-1.806230,3.404908,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.011398,1.031173,-1.472984,1.165489,-1.225534,1.142737,1.043480
190,1.303572,1.337339,-1.727496,3.671303,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.028853,1.025017,-1.478716,1.158547,-1.211200,1.137574,1.038106


In [20]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      24.166
1      21.774
2      28.228
3      25.525
4      26.207
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Alagoas - Consumo de Cimento (t), Length: 240, dtype: float64

In [21]:
# input para treinamento
train_input = input_data.iloc[:train_split]
train_input

,Unnamed: 0,Alagoas - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Alagoas - value,Alagoas - Desemprego,Alagoas - Produção de Cimento (t),Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado
0,-1.593255,-2.106195,2.723741,0.364808,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.400856,-0.877266,-0.885888,-1.644358,-1.297696,-1.913990,-1.857572
1,-1.303572,-2.072296,2.350880,0.368956,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.374741,-0.879298,-0.900149,-1.627544,-1.242142,-1.888746,-1.832102
2,-1.013890,-2.038397,2.123016,0.331110,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.355481,-0.881330,-0.904358,-1.610731,-1.186588,-1.863502,-1.806633
3,-0.724207,-2.004498,2.021477,0.244570,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.337169,-0.883362,-0.917668,-1.593917,-1.131034,-1.838257,-1.781163
4,-0.434524,-1.970599,1.887113,0.279013,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.319844,-0.885394,-0.910766,-1.577104,-1.075480,-1.813013,-1.755694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,-1.303572,1.429306,-0.214006,1.216560,0.059280,-1.233012,1.031384,0.819304,-0.883659,1.348585,1.320458,-0.588269,1.217771,-1.179058,1.109023,1.141942
158,-1.013890,1.428089,-0.434717,1.155373,0.222389,-1.299304,1.042716,0.808136,-0.950771,1.330705,1.307307,-0.687207,1.221600,-1.195069,1.116989,1.144540
159,-0.724207,1.426872,-0.524091,1.056140,0.191929,-1.248662,1.054049,0.796969,-1.028465,1.311609,1.294156,-0.785508,1.225429,-1.211080,1.124956,1.147138
160,-0.434524,1.425655,-0.614500,1.147619,0.385687,-1.068274,1.065381,0.785801,-1.103668,1.291203,1.281004,-0.883126,1.229259,-1.227092,1.132922,1.149736


In [22]:
# Alvo para treinamento
train_target = target_data.iloc[:train_split]
train_target

0      24.166
1      21.774
2      28.228
3      25.525
4      26.207
        ...  
157    32.321
158    41.937
159    31.982
160    28.499
161    22.626
Name: Alagoas - Consumo de Cimento (t), Length: 162, dtype: float64

In [23]:
#input de test (Ano 2021)
test_input = input_data.iloc[train_split:]
test_input

,Unnamed: 0,Alagoas - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Alagoas - value,Alagoas - Desemprego,Alagoas - Produção de Cimento (t),Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado
162,0.144841,1.423220,-0.601510,1.044342,0.157879,-1.255326,1.088045,0.763466,-1.213929,1.247022,1.254702,-1.076156,1.236917,-1.259114,1.148855,1.154932
163,0.434524,1.422003,-0.786068,1.062406,0.548855,-1.118679,1.099377,0.752299,-1.292173,1.223337,1.241551,-1.171477,1.240746,-1.275126,1.156821,1.157530
164,0.724207,1.420786,-0.830387,0.969885,0.385685,-0.936336,1.110709,0.741131,-1.324219,1.204693,1.228399,-1.265945,1.244576,-1.291137,1.164788,1.160128
165,1.013890,1.419568,-0.801089,0.801374,0.420965,-0.931790,1.122042,0.729964,-1.344446,1.185204,1.215248,-1.336584,1.248405,-1.307148,1.172754,1.162726
166,1.303572,1.418351,-0.959917,1.125021,0.548870,-1.168522,1.133374,0.718796,-1.381638,1.170986,1.202097,-1.406157,1.252234,-1.323160,1.180720,1.165324
167,1.593255,1.417134,-1.022309,1.148821,0.222499,-1.285611,1.144706,0.707629,-1.411208,1.156220,1.188946,-1.475508,1.256063,-1.339171,1.188687,1.167922
168,-1.593255,1.415917,-1.074401,0.446496,-0.614754,-1.446316,1.156038,0.696461,-1.412953,1.140972,1.175794,-1.532489,1.259893,-1.355182,1.196653,1.170520
169,-1.303572,1.415213,-1.119597,0.726189,-0.478229,-1.357832,1.167287,0.681823,-1.491464,1.128313,1.168359,-1.592879,1.257232,-1.355129,1.196033,1.163964
170,-1.013890,1.414510,-1.078648,0.597090,-0.393157,-1.235682,1.178536,0.667184,-1.573805,1.115315,1.160924,-1.642238,1.254572,-1.355076,1.195413,1.157408
171,-0.724207,1.413807,-1.055426,0.457269,-0.557552,-1.065340,1.189784,0.652545,-1.564950,1.101989,1.153488,-1.707022,1.251912,-1.355023,1.194793,1.150852


In [24]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[train_split:split_index + 1]
test_target

162    21.340
163    26.848
164    28.074
165    29.941
166    28.977
167    31.580
168    29.991
169    26.820
170    29.446
171    27.666
172    22.013
173    28.025
174    25.648
175    31.782
176    27.768
177    33.073
178    30.389
179    27.657
180    32.388
181    30.378
182    30.641
183    32.337
184    36.405
185    22.499
186    27.961
187    32.378
188    32.582
189    42.703
190    38.584
191    36.503
Name: Alagoas - Consumo de Cimento (t), dtype: float64

In [25]:
def validation_splitter(df, div_factor):
    split_factor = len(df)//div_factor
    positions_to_drop = []
    for i in range(split_factor//2):
        pos = df.shape[0] - (i*div_factor + 1)
        positions_to_drop.append(pos)
    
    return df.drop(positions_to_drop), df.iloc[positions_to_drop]

In [26]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
#     train, train_val = validation_splitter(train_input, 6)
#     target,target_val = validation_splitter(train_target, 6)
#     display(train)
#     display(train_val)
#     display(target)
#     display(target_val)
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
#                         validation_data=(train_val,target_val),
                        validation_split=0.1,
                        callbacks=[early_stopping],
                        verbose=want_verbose)
    return model, history

In [27]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(25)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [28]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[276696894, 2014889265, 372569930, 2677425768, 1919255007, 1911378343, 2485568703, 217687616, 43295332, 3998873420, 3563295006, 3367150844, 256132084, 2691593933, 1992719647, 1480873956, 2148446215, 4239312448, 2461111171, 3682730183, 2977149366, 1766794138, 172313718, 1546819712, 3006489405]


Step: 0 ___________________________________________
val_loss: 767.68212890625
winner_seed: 276696894


Step: 1 ___________________________________________
val_loss: 787.0926513671875


Step: 2 ___________________________________________
val_loss: 2554.7939453125


Step: 3 ___________________________________________
val_loss: 1716.1842041015625


Step: 4 ___________________________________________
val_loss: 584.2723388671875
winner_seed: 1919255007


Step: 5 ___________________________________________
val_loss: 1471.868408203125


Step: 6 ___________________________________________
val_loss: 192.3546142578125
winner_seed: 2485568703


Step: 7 ___________________________________________
val_loss: 

In [29]:
trained_model, history = neural_network_model(train_input, 
                                              train_target, 
                                              want_verbose=1, 
                                              seed=winner_seed)

Epoch 1/10000
5/5 [==============================] - 1s 26ms/step - loss: 2160.5251 - val_loss: 966.0403
Epoch 2/10000
5/5 [==============================] - 0s 6ms/step - loss: 1885.2291 - val_loss: 800.0574
Epoch 3/10000
5/5 [==============================] - 0s 5ms/step - loss: 1914.3767 - val_loss: 3696.5188
Epoch 4/10000
5/5 [==============================] - 0s 6ms/step - loss: 1949.6500 - val_loss: 582.2033
Epoch 5/10000
5/5 [==============================] - 0s 5ms/step - loss: 1777.3407 - val_loss: 1540.2070
Epoch 6/10000
5/5 [==============================] - 0s 5ms/step - loss: 1607.2922 - val_loss: 1658.3398
Epoch 7/10000
5/5 [==============================] - 0s 5ms/step - loss: 1532.3418 - val_loss: 560.2144
Epoch 8/10000
5/5 [==============================] - 0s 5ms/step - loss: 1441.5433 - val_loss: 620.0351
Epoch 9/10000
5/5 [==============================] - 0s 5ms/step - loss: 1285.3163 - val_loss: 501.8520
Epoch 10/10000
5/5 [==============================] - 0s 5ms

5/5 [==============================] - 0s 5ms/step - loss: 27.6202 - val_loss: 43.5069
Epoch 81/10000
5/5 [==============================] - 0s 5ms/step - loss: 24.0942 - val_loss: 42.0681
Epoch 82/10000
5/5 [==============================] - 0s 5ms/step - loss: 25.6258 - val_loss: 35.8479
Epoch 83/10000
5/5 [==============================] - 0s 5ms/step - loss: 24.6340 - val_loss: 34.0327
Epoch 84/10000
5/5 [==============================] - 0s 5ms/step - loss: 25.7438 - val_loss: 35.1052
Epoch 85/10000
5/5 [==============================] - 0s 5ms/step - loss: 27.1398 - val_loss: 37.1021
Epoch 86/10000
5/5 [==============================] - 0s 5ms/step - loss: 24.1073 - val_loss: 34.0886
Epoch 87/10000
5/5 [==============================] - 0s 5ms/step - loss: 21.6320 - val_loss: 31.5799
Epoch 88/10000
5/5 [==============================] - 0s 5ms/step - loss: 23.1567 - val_loss: 31.9929
Epoch 89/10000
5/5 [==============================] - 0s 6ms/step - loss: 24.3922 - val_loss: 35.

Epoch 160/10000
5/5 [==============================] - 0s 6ms/step - loss: 22.3227 - val_loss: 38.2421
Epoch 161/10000
5/5 [==============================] - 0s 5ms/step - loss: 20.6779 - val_loss: 35.5881
Epoch 162/10000
5/5 [==============================] - 0s 5ms/step - loss: 21.2097 - val_loss: 33.8579
Epoch 163/10000
5/5 [==============================] - 0s 5ms/step - loss: 19.4817 - val_loss: 33.8033
Epoch 164/10000
5/5 [==============================] - 0s 5ms/step - loss: 19.2561 - val_loss: 28.9213
Epoch 165/10000
5/5 [==============================] - 0s 5ms/step - loss: 21.6027 - val_loss: 29.7859
Epoch 166/10000
5/5 [==============================] - 0s 6ms/step - loss: 20.5319 - val_loss: 27.5515
Epoch 167/10000
5/5 [==============================] - 0s 5ms/step - loss: 20.7432 - val_loss: 29.9130
Epoch 168/10000
5/5 [==============================] - 0s 5ms/step - loss: 22.1251 - val_loss: 38.7150
Epoch 169/10000
5/5 [==============================] - 0s 5ms/step - loss

5/5 [==============================] - 0s 5ms/step - loss: 21.3869 - val_loss: 32.6103
Epoch 240/10000
5/5 [==============================] - 0s 5ms/step - loss: 20.5779 - val_loss: 39.1323
Epoch 241/10000
5/5 [==============================] - 0s 5ms/step - loss: 20.1023 - val_loss: 35.4748
Epoch 242/10000
5/5 [==============================] - 0s 5ms/step - loss: 21.7442 - val_loss: 28.8393
Epoch 243/10000
5/5 [==============================] - 0s 5ms/step - loss: 19.0493 - val_loss: 28.1407
Epoch 244/10000
5/5 [==============================] - 0s 5ms/step - loss: 24.4929 - val_loss: 35.4423
Epoch 245/10000
5/5 [==============================] - 0s 5ms/step - loss: 21.5930 - val_loss: 28.6513
Epoch 246/10000
5/5 [==============================] - 0s 5ms/step - loss: 21.3804 - val_loss: 30.7446
Epoch 247/10000
5/5 [==============================] - 0s 5ms/step - loss: 20.2144 - val_loss: 46.4002
Epoch 248/10000
5/5 [==============================] - 0s 5ms/step - loss: 20.5690 - val_

5/5 [==============================] - 0s 6ms/step - loss: 19.8984 - val_loss: 43.0063
Epoch 319/10000
5/5 [==============================] - 0s 5ms/step - loss: 19.4037 - val_loss: 36.1003
Epoch 320/10000
5/5 [==============================] - 0s 5ms/step - loss: 24.7701 - val_loss: 45.6797
Epoch 321/10000
5/5 [==============================] - 0s 5ms/step - loss: 20.5462 - val_loss: 37.2906
Epoch 322/10000
5/5 [==============================] - 0s 6ms/step - loss: 19.1771 - val_loss: 49.8198
Epoch 323/10000
5/5 [==============================] - 0s 5ms/step - loss: 19.9359 - val_loss: 37.3133
Epoch 324/10000
5/5 [==============================] - 0s 5ms/step - loss: 18.5870 - val_loss: 30.0907
Epoch 325/10000
5/5 [==============================] - 0s 5ms/step - loss: 19.5342 - val_loss: 31.8139
Epoch 326/10000
5/5 [==============================] - 0s 6ms/step - loss: 18.3476 - val_loss: 32.0887
Epoch 327/10000
5/5 [==============================] - 0s 6ms/step - loss: 18.9936 - val_

...

5/5 [==============================] - 0s 4ms/step - loss: 11.9837 - val_loss: 36.0888
Epoch 951/10000
5/5 [==============================] - 0s 5ms/step - loss: 11.8174 - val_loss: 52.9515
Epoch 952/10000
5/5 [==============================] - 0s 5ms/step - loss: 12.7975 - val_loss: 34.8704
Epoch 953/10000
5/5 [==============================] - 0s 5ms/step - loss: 12.6518 - val_loss: 54.9135
Epoch 954/10000
5/5 [==============================] - 0s 5ms/step - loss: 13.1523 - val_loss: 40.1852
Epoch 955/10000
5/5 [==============================] - 0s 5ms/step - loss: 12.4376 - val_loss: 35.4770
Epoch 956/10000
5/5 [==============================] - 0s 6ms/step - loss: 14.2072 - val_loss: 48.3078
Epoch 957/10000
5/5 [==============================] - 0s 5ms/step - loss: 13.8324 - val_loss: 35.2149
Epoch 958/10000
5/5 [==============================] - 0s 5ms/step - loss: 13.4197 - val_loss: 36.6208
Epoch 959/10000
5/5 [==============================] - 0s 4ms/step - loss: 13.4484 - val_

5/5 [==============================] - 0s 5ms/step - loss: 12.3645 - val_loss: 38.4567
Epoch 1030/10000
5/5 [==============================] - 0s 5ms/step - loss: 12.8568 - val_loss: 45.7148
Epoch 1031/10000
5/5 [==============================] - 0s 5ms/step - loss: 11.4478 - val_loss: 45.0768
Epoch 1032/10000
5/5 [==============================] - 0s 5ms/step - loss: 11.0843 - val_loss: 37.7155
Epoch 1033/10000
5/5 [==============================] - 0s 5ms/step - loss: 12.4827 - val_loss: 43.9320
Epoch 1034/10000
5/5 [==============================] - 0s 5ms/step - loss: 11.4418 - val_loss: 38.2197
Epoch 1035/10000
5/5 [==============================] - 0s 5ms/step - loss: 12.5483 - val_loss: 47.4809
Epoch 1036/10000
5/5 [==============================] - 0s 5ms/step - loss: 11.3927 - val_loss: 44.4387
Epoch 1037/10000
5/5 [==============================] - 0s 5ms/step - loss: 11.9713 - val_loss: 48.2355
Epoch 1038/10000
5/5 [==============================] - 0s 5ms/step - loss: 10.28

5/5 [==============================] - 0s 5ms/step - loss: 10.1145 - val_loss: 46.2033
Epoch 1109/10000
5/5 [==============================] - 0s 5ms/step - loss: 10.8296 - val_loss: 48.6383
Epoch 1110/10000
5/5 [==============================] - 0s 5ms/step - loss: 10.4243 - val_loss: 41.2875
Epoch 1111/10000
5/5 [==============================] - 0s 5ms/step - loss: 10.5904 - val_loss: 44.6231
Epoch 1112/10000
5/5 [==============================] - 0s 5ms/step - loss: 10.0886 - val_loss: 51.5055
Epoch 1113/10000
5/5 [==============================] - 0s 5ms/step - loss: 10.7687 - val_loss: 43.9396
Epoch 1114/10000
5/5 [==============================] - 0s 4ms/step - loss: 9.4321 - val_loss: 53.0482
Epoch 1115/10000
5/5 [==============================] - 0s 5ms/step - loss: 9.9545 - val_loss: 45.1691
Epoch 1116/10000
5/5 [==============================] - 0s 5ms/step - loss: 9.6993 - val_loss: 47.4031
Epoch 1117/10000
5/5 [==============================] - 0s 5ms/step - loss: 8.6092 -

5/5 [==============================] - 0s 5ms/step - loss: 8.9401 - val_loss: 57.2324
Epoch 1188/10000
5/5 [==============================] - 0s 5ms/step - loss: 11.5649 - val_loss: 44.3685
Epoch 1189/10000
5/5 [==============================] - 0s 5ms/step - loss: 10.2015 - val_loss: 55.1736
Epoch 1190/10000
5/5 [==============================] - 0s 4ms/step - loss: 9.8200 - val_loss: 55.0418
Epoch 1191/10000
5/5 [==============================] - 0s 4ms/step - loss: 10.6164 - val_loss: 48.5350
Epoch 1192/10000
5/5 [==============================] - 0s 5ms/step - loss: 9.8392 - val_loss: 54.4326
Epoch 1193/10000
5/5 [==============================] - 0s 5ms/step - loss: 10.8234 - val_loss: 54.2729
Epoch 1194/10000
5/5 [==============================] - 0s 4ms/step - loss: 10.0325 - val_loss: 52.2374
Epoch 1195/10000
5/5 [==============================] - 0s 4ms/step - loss: 9.4366 - val_loss: 55.2501
Epoch 1196/10000
5/5 [==============================] - 0s 5ms/step - loss: 9.9821 - 

5/5 [==============================] - 0s 5ms/step - loss: 8.2683 - val_loss: 59.0172
Epoch 1267/10000
5/5 [==============================] - 0s 5ms/step - loss: 9.3082 - val_loss: 48.9166
Epoch 1268/10000
5/5 [==============================] - 0s 5ms/step - loss: 8.1211 - val_loss: 62.6254
Epoch 1269/10000
5/5 [==============================] - 0s 5ms/step - loss: 8.2593 - val_loss: 59.4994
Epoch 1270/10000
5/5 [==============================] - 0s 4ms/step - loss: 7.8432 - val_loss: 50.8547
Epoch 1271/10000
5/5 [==============================] - 0s 4ms/step - loss: 7.9450 - val_loss: 54.4274
Epoch 1272/10000
5/5 [==============================] - 0s 5ms/step - loss: 9.2281 - val_loss: 56.5371
Epoch 1273/10000
5/5 [==============================] - 0s 5ms/step - loss: 8.7751 - val_loss: 50.8791
Epoch 1274/10000
5/5 [==============================] - 0s 5ms/step - loss: 8.8726 - val_loss: 57.6260
Epoch 1275/10000
5/5 [==============================] - 0s 5ms/step - loss: 8.8995 - val_l

In [30]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input.iloc[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [31]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            test_input, 
                                            test_target, 
                                            train_split)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 14ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,31.604759,30.968718,30.79285,30.783115,30.775574,30.753244,30.778139,30.77615,30.777201,30.777308,30.775316,30.778269,30.775528,30.772388,30.775427,30.775833,30.775751,28.633604,30.779169,30.776634,30.77688,30.774754,24.819443,22.558033,22.479984,22.459362,22.452263,22.450525,22.44672,22.443668
Target,21.34,26.848,28.074,29.941,28.977,31.58,29.991,26.82,29.446,27.666,22.013,28.025,25.648,31.782,27.768,33.073,30.389,27.657,32.388,30.378,30.641,32.337,36.405,22.499,27.961,32.378,32.582,42.703,38.584,36.503
Error,10.264759,4.120718,2.718851,0.842115,1.798574,0.826756,0.78714,3.95615,1.331202,3.111307,8.762316,2.753269,5.127527,1.009611,3.007427,2.297169,0.386751,0.976604,1.608831,0.398634,0.13588,1.562248,11.585556,0.059032,5.481016,9.918636,10.129738,20.252474,16.13728,14.05933


In [32]:
display(mae)
display(mape)

4.846896

0.15534142

In [33]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [34]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             train_split)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 13ms/step
Ano-0: |Prediction[[370.34067]] - Target[330.72099999999995]| =  Error: [[39.61966]]; MAPE:[[0.11979783]]
1/1 [==============================] - 0s 13ms/step
Ano-0: |Prediction[[352.99347]] - Target[360.96500000000003]| =  Error: [[7.971527]]; MAPE:[[0.02208393]]
1/1 [==============================] - 0s 13ms/step
Ano-5: |Prediction[[134.73251]] - Target[210.711]| =  Error: [[75.978485]]; MAPE:[[0.3605815]]


[array([[39.61966]], dtype=float32),
 array([[7.971527]], dtype=float32),
 array([[75.978485]], dtype=float32)]

41.18989

0.16748774